In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from scipy import io
from tensorflow.keras.callbacks import ModelCheckpoint

In [8]:
dataset = io.loadmat('datasets/emnist-letters.mat')
# matlab_structs / numpy structured arrays. yay
# they have a dtype (structure) and every element adheres to it :)

In [9]:
struct_dataset = dataset['dataset'][0,0]

train_images, train_labels = struct_dataset['train'][0,0]['images'], struct_dataset['train'][0,0]['labels']
test_images, test_labels = struct_dataset['test'][0,0]['images'], struct_dataset['test'][0,0]['labels']

print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)

(124800, 784) (20800, 784) (124800, 1) (20800, 1)


In [10]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding="same", use_bias="false", input_shape=(28,28,1)),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.35),
    layers.Dense(26, activation="softmax")
])

model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/opt/anaconda3/envs/testenv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = train_labels - 1
test_labels = test_labels - 1

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

checkpoint = ModelCheckpoint(filepath="checkpoints/epoch_{epoch:02d}.keras", save_weights_only=False, save_freq="epoch")

model.fit(train_images, train_labels, epochs=15,
          validation_data=(test_images, test_labels), callbacks=[checkpoint], batch_size=64)

# Best = 95.13% validation acc

In [11]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = train_labels - 1
test_labels = test_labels - 1

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

model = tf.keras.models.load_model("alpha_identifier.keras")
loss, acc = model.evaluate(test_images, test_labels, batch_size=64)
print("Accuracy: ", acc*100, " Loss: ", loss)

325/325 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9513 - loss: 0.1702
Accuracy:  95.12500166893005  Loss:  0.17023393511772156
